In [27]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [28]:
from tensorflow.keras.layers import Input, Conv1D
from tensorflow.keras import Model
import numpy as np

In [29]:
inputs = Input(shape=(20,3))
x = Conv1D(2,4,3)(inputs)
model = Model(inputs, x)

In [30]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 20, 3)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 6, 2)           │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 26 (104.00 B)

 Trainable params: 26 (104.00 B)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.weights

[<KerasVariable shape=(4, 3, 2), dtype=float32, path=conv1d_2/kernel>,
 <KerasVariable shape=(2,), dtype=float32, path=conv1d_2/bias>]

In [32]:
X = np.random.rand(1,20,3)
X

array([[[0.77121003, 0.42941302, 0.24685043],
        [0.72466623, 0.0257568 , 0.77939671],
        [0.70648664, 0.05427524, 0.88182928],
        [0.23631579, 0.289313  , 0.43766315],
        [0.24496628, 0.82382481, 0.6841791 ],
        [0.54075838, 0.54196626, 0.932303  ],
        [0.26840549, 0.34903981, 0.26140071],
        [0.19027901, 0.66469758, 0.47326783],
        [0.04876403, 0.94405817, 0.6470154 ],
        [0.10220359, 0.34710854, 0.06408549],
        [0.94281552, 0.26884967, 0.09277842],
        [0.81799077, 0.09165016, 0.23117276],
        [0.68465778, 0.72629515, 0.38909141],
        [0.96990681, 0.77781652, 0.14598749],
        [0.59614585, 0.38079966, 0.72758417],
        [0.65319074, 0.85866742, 0.84935527],
        [0.97527495, 0.85598694, 0.60793867],
        [0.05568378, 0.16019932, 0.98967926],
        [0.28394094, 0.85454243, 0.31499458],
        [0.34925732, 0.80400813, 0.52813214]]])

In [33]:
y = model.predict(X)
y

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


array([[[-0.3385666 ,  0.04850817],
        [ 0.51277256,  0.644366  ],
        [ 0.3746652 ,  0.49716341],
        [-0.17809692,  0.92168564],
        [-0.01779684,  1.0714066 ],
        [-0.3292515 ,  1.0170866 ]]], dtype=float32)

In [34]:
X_in = [[int(X[0][i][j]*2**10) for j in range(3)] for i in range(20)]
X_in_unchanged = [[str(X[0][i][j]) for j in range(3)] for i in range(20)]
weights = [[[int(model.weights[0].numpy()[i][j][k]*2**10) for k in range(2)] for j in range(3)] for i in range(4)]
weights_unchanged = [[[str(model.weights[0].numpy()[i][j][k]) for k in range(2)] for j in range(3)] for i in range(4)]
bias = [int(model.weights[1].numpy()[i]*2**20) for i in range(2)]
bias_unchanged = [str(model.weights[1].numpy()[i]) for i in range(2)]

In [35]:
def Conv1DInt(nInputs, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    Input = [[str(input[i][j]) for j in range(nChannels)] for i in range(nInputs)]
    Weights = [[[str(weights[i][j][k]) for k in range(nFilters)] for j in range(nChannels)] for i in range(kernelSize)]
    Bias = [str(bias[i]) for i in range(nFilters)]
    out = [[0 for _ in range(nFilters)] for j in range((nInputs - kernelSize)//strides + 1)]
    for i in range((nInputs - kernelSize)//strides + 1):
        for j in range(nFilters):
            for k in range(kernelSize):
                for l in range(nChannels):
                    out[i][j] += input[i*strides + k][l]*weights[k][l][j]
            out[i][j] += bias[j]
            out[i][j] = str(out[i][j] // n)
    return Input, Weights, Bias, out

In [36]:
X_in, weights, bias, out = Conv1DInt(20, 3, 2, 4, 3, 2**10, X_in, weights, bias)
out

[['-347', '48'],
 ['524', '658'],
 ['382', '507'],
 ['-183', '941'],
 ['-19', '1093'],
 ['-337', '1039']]

In [37]:
in_json = {
    "in": X_in,
    "weights": weights,
    "bias": bias,
    "out": out,
    "in_unchanged": X_in_unchanged,
    "weights_unchanged": weights_unchanged,
    "bias_unchanged": bias_unchanged
}

In [38]:
import json

In [39]:
with open("conv1D_input.json", "w") as f:
    json.dump(in_json, f)